# Рекомендательные системы

## Урок 6. Двухуровневые модели рекомендаций

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import bsr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender

from lightgbm import LGBMClassifier

In [2]:
RETAIL_DATA = "../hw2/retail_train.csv.zip"
PRODUCT_DATA = "../hw2/product.csv"
DEMOGRAPHIC_DATA = "../hw5/hh_demographic.csv"
VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [3]:
ITEM_ID_COL = 'item_id'
USER_ID_COL = 'user_id'
ITEM_INDEX_COL = 'item_idx'
USER_INDEX_COL = 'user_idx'
WEEK_NUM_COL = 'week_no'

### Библиотека

In [4]:
# Precision@K
def precision_at_k(recommended_list, bought_list, k=5):
    try:
        _rec_list = recommended_list[:k]
        _b_and_r = np.intersect1d(bought_list, _rec_list)
        return _b_and_r.size / len(_rec_list)
    except (ZeroDivisionError, TypeError):
        return 0.0

def mean_precision_at_k(df, rec, bought, k=5):
    _result = df.apply(
        lambda row: precision_at_k(row[rec], row[bought], k),
        axis=1
    )
    return np.mean(_result)

In [5]:
# Recall@K
def recall_at_k(recommended_list, bought_list, k=5):
    try:
        _rec_list = recommended_list[:k]
        _b_and_r = np.intersect1d(bought_list, _rec_list)
        return _b_and_r.size / len(bought_list)
    except (ZeroDivisionError, TypeError):
        return 0.0

def mean_recall_at_k(df, rec, bought, k=5):
    _result = df.apply(
        lambda row: recall_at_k(row[rec], row[bought], k),
        axis=1
    )
    return np.mean(_result)

In [6]:
def read_transform_csv(path, column_map={}, index=None, **kwargs):
    columns = pd.read_csv(path, nrows=0).columns
    _column_map = dict(zip(columns, columns.str.lower()))
    _column_map.update(column_map)
    _data = pd.read_csv(path, **kwargs).rename(columns=_column_map)
    if index is not None:
        return _data.set_index(index)    
    return _data

In [7]:
# Предфильтрация
def prefilter_items(data, prevalence_range = (0.05, 0.95), price_range = (1.0, 100.0)):
    # Уберем самые популярные товары и самые непопулярные товары
    pop_thr, unpop_thr = prevalence_range
    item_cum_counts = data[ITEM_ID_COL].value_counts().cumsum()
    max_count = item_cum_counts.values[-1]
    top_popular_mask = item_cum_counts < max_count * pop_thr
    top_uppopular_mask = item_cum_counts > max_count * unpop_thr
    blocked_items = item_cum_counts[top_popular_mask | top_uppopular_mask].index
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    recent_sale_items = data[ITEM_ID_COL][data[WEEK_NUM_COL] > data[WEEK_NUM_COL].max() - 53]
    old_sale_items = np.setdiff1d(data[ITEM_ID_COL], recent_sale_items)
    blocked_items = np.union1d(blocked_items, old_sale_items)
    
    # Уберем слишком дешевые товары и слишком дорогие товары
    # Цена товара косвенно оценивается по sales_value
    min_price, max_price = price_range
    bad_price_items = (
        data
        .assign(price = lambda x: np.where(x['quantity'] > 0, x['sales_value'] / x['quantity'], 0.0))
        .groupby(ITEM_ID_COL)
        .agg(min_item_price=('price', 'min'), max_item_price=('price', 'max'))
        .query("min_item_price >= @max_price or max_item_price <= @min_price")
        .index
    )
    blocked_items = np.union1d(blocked_items, bad_price_items)
    return data[~np.isin(data[ITEM_ID_COL], blocked_items)].copy()

In [8]:
# RecStore - класс, аккумулирующий результаты работы разных рекомендательных моделей
class RecStore:
    _metric_dispatcher = {
        'recall': mean_recall_at_k,
        'precision': mean_precision_at_k
    }
    
    def __init__(self, data, aggcol, aggfunc):  # training data
        agg_df = (
            data.groupby([USER_ID_COL, ITEM_ID_COL])
            .agg(interaction=(aggcol, aggfunc))
            .reset_index()
        )
        self.user_idx_id = pd.DataFrame(enumerate(np.sort(agg_df[USER_ID_COL].unique())), 
                                        columns=[USER_INDEX_COL, USER_ID_COL])
        self.item_idx_id = pd.DataFrame(enumerate(np.sort(agg_df[ITEM_ID_COL].unique())), 
                                        columns=[ITEM_INDEX_COL, ITEM_ID_COL])
        self.user_item_interaction = (
            agg_df
            .merge(self.user_idx_id, on=USER_ID_COL)
            .merge(self.item_idx_id, on=ITEM_ID_COL)
        )
        interaction = self.user_item_interaction['interaction'].astype(float)
        user_idx = self.user_item_interaction[USER_INDEX_COL]
        item_idx = self.user_item_interaction[ITEM_INDEX_COL]
        self.user_item_matrix = bsr_matrix((interaction, (user_idx, item_idx)), 
                                           shape=(user_idx.max()+1, item_idx.max()+1)).tocsr()
        self.item_popularity = (
            self.user_item_interaction
            .groupby(ITEM_ID_COL)
            .agg({'interaction': 'sum'})
            .sort_values('interaction', ascending=False)
            .reset_index()
        )
        self.recommendations = self.user_idx_id
        self.actuals = []
    
    def add_actual(self, data, name=None):  # test data
        if name is None:
            name = ACTUAL_COL
        self.actuals.append(
            data[[USER_ID_COL, ITEM_ID_COL]]
            .drop_duplicates()
            .groupby(USER_ID_COL)
            .agg({ITEM_ID_COL: list})
            .rename(columns={ITEM_ID_COL: name})
        )
    
    @property
    def result(self):
        _result = self.recommendations
        for actual_df in self.actuals:
            _result = _result.merge(actual_df, on=USER_ID_COL, how="left")
        return _result
    
    def _get_recommendations(self, model, userid, N):
        model_rec = model.recommend(userid, self.user_item_matrix, N=N,
                              filter_already_liked_items=False,
                              recalculate_user=False)
        rec_df = pd.DataFrame(model_rec, columns=[ITEM_INDEX_COL, 'model_score']).set_index(ITEM_INDEX_COL)
        rec = rec_df.join(self.item_idx_id)[ITEM_ID_COL]
        return rec.tolist()
    
    def _extend_rec_with_popular(self, lst, N):
        if isinstance(lst, list):
            if len(lst) == N:
                return lst            
            else:
                series = pd.Series(lst).append(self.item_popularity[ITEM_ID_COL]).unique()[:N]
        else:
            series = self.item_popularity[ITEM_ID_COL][:N]
        return series.tolist()

    def add_recommendations(self, model, N=5, model_name=None, extend_with_popular=True, show_progress=False):
        if model_name is None:
            model_name = f"{model.__class__.__name__}, N={N}"
        
        model.fit(self.user_item_matrix.T, show_progress=show_progress)
        if hasattr(model, 'user_factors') and hasattr(model, 'item_factors'):
            fast_recs = model.user_factors @ model.item_factors.T
            item_ids = self.item_idx_id[ITEM_ID_COL].values
            rec_matrix = item_ids[np.argsort(-fast_recs)[:, :N]]
            rec_df = pd.DataFrame.from_records(
                np.expand_dims(rec_matrix, axis=1),
                columns = [model_name],
                index=self.user_idx_id[USER_INDEX_COL]
            )
            self.recommendations = self.recommendations.join(rec_df)
        else:        
            def _get_user_rec(userid):
                return self._get_recommendations(model=model, userid=userid, N=N) 
        
            self.recommendations[model_name] = self.recommendations[USER_INDEX_COL].apply(_get_user_rec)
        if extend_with_popular:
            def _extend_with_popular(lst):
                return self._extend_rec_with_popular(lst, N=N)
            
            self.recommendations[model_name] = self.recommendations[model_name].apply(_extend_with_popular)
        return self
    
    def rerank(self, model_name, user_item_proba, suffix=', reranked'):
        rec_reranked = (
            self.recommendations
            .filter([USER_ID_COL, model_name])
            .rename(columns={model_name: ITEM_ID_COL})
            .explode(ITEM_ID_COL)
            .assign(orig_rank = lambda x: x.groupby(USER_ID_COL).cumcount())
            .merge(user_item_proba, on=[USER_ID_COL, ITEM_ID_COL], how='left')
            .assign(sort_factor = lambda x: np.where(x.proba.isna(), x.orig_rank, -x.proba))
            .sort_values([USER_ID_COL, 'sort_factor'], ascending=True)
            .groupby(USER_ID_COL)
            .agg({ITEM_ID_COL: list})
            .rename(columns={ITEM_ID_COL: f"{model_name}{suffix}"})
        )
        self.recommendations = self.recommendations.merge(rec_reranked, on=USER_ID_COL, how="left")
    
    def get_metric_table(self, actual, metrics, dropna=True):
        if isinstance(metrics, str):
            metrics = [metrics]
        metric_holder = {m: [] for m in metrics}
        model_names = rs.recommendations.columns[2:].tolist()
        _result = rs.result.filter(model_names + [actual])
        if dropna:
            _result = _result.dropna()
        for metric in metrics:
            metric_N = metric.lower().split('@')
            metric_func_key = metric_N[0]
            if metric_func_key not in self._metric_dispatcher:
                continue 
            metric_func = self._metric_dispatcher[metric_func_key]
            kwargs = {}
            if len(metric_N) > 1 and metric_N[1].isdigit():
                kwargs['k'] = int(metric_N[1])
            for model_name in model_names:
                metric_holder[metric].append(metric_func(_result, model_name, actual, **kwargs))
        return pd.DataFrame(metric_holder, index=model_names)

**Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


### Данные

In [9]:
%%time
item_features = read_transform_csv(PRODUCT_DATA, {'PRODUCT_ID': ITEM_ID_COL}, index=ITEM_ID_COL, 
                                   na_values=['Unknown', 'None/Unknown'])
user_features = read_transform_csv(DEMOGRAPHIC_DATA, {'household_key': USER_ID_COL}, index=USER_ID_COL,
                                  na_values=['Unknown', 'None/Unknown'])
# train test split
data = pd.read_csv(RETAIL_DATA)
# берем данные для тренировки matching модели
data_train_matcher = data[data[WEEK_NUM_COL] < data[WEEK_NUM_COL].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]
# берем данные для валидации matching модели
data_val_matcher = data[(data[WEEK_NUM_COL] >= data[WEEK_NUM_COL].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data[WEEK_NUM_COL] < data[WEEK_NUM_COL].max() - (VAL_RANKER_WEEKS))]
# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
# берем данные для теста ranking, matching модели
data_val_ranker = data[data[WEEK_NUM_COL] >= data[WEEK_NUM_COL].max() - VAL_RANKER_WEEKS]

# Prefiltered
data_train_matcher_filtered = prefilter_items(data_train_matcher, prevalence_range = (0.05, 0.75), price_range = (1.8, 50.0))
print(f"Decreased # of items from {data_train_matcher[ITEM_ID_COL].nunique()}"
      f" to {data_train_matcher_filtered[ITEM_ID_COL].nunique()}")

Decreased # of items from 83685 to 5426
Wall time: 5.57 s


In [10]:
# Result Accumulator
rs = RecStore(data_train_matcher_filtered, 'quantity', 'count')
rs.add_actual(data_val_matcher, 'matcher_actual')
rs.add_actual(data_val_ranker, 'ranker_actual')

In [11]:
%%time
models = [
    ("Own Rec", ItemItemRecommender(K=1, num_threads=4)),
    ("ALS", AlternatingLeastSquares(factors=3, regularization=0.05, iterations=10))
]

for k in (20, 50, 100, 200, 500):
    for model_name, model in models:
        model_name=f"{model_name}, N={k}"
        rs.add_recommendations(model, N=k, model_name=model_name, extend_with_popular=True)

Wall time: 47.8 s


In [12]:
%%time
rs.get_metric_table('matcher_actual', ['recall@20', 'recall@50', 'recall@100', 'recall@200', 
                                      'precision@5', 'precision@10', 'precision@50'])

Wall time: 15.3 s


,recall@20,recall@50,recall@100,recall@200,precision@5,precision@10,precision@50
"Own Rec, N=20",0.053468,0.053468,0.053468,0.053468,0.243584,0.195427,0.152193
"ALS, N=20",0.026949,0.026949,0.026949,0.026949,0.078488,0.080495,0.076552
"Own Rec, N=50",0.053468,0.080064,0.080064,0.080064,0.243584,0.195427,0.099907
"ALS, N=50",0.026949,0.048537,0.048537,0.048537,0.078488,0.080495,0.059907
"Own Rec, N=100",0.053468,0.080064,0.100154,0.100154,0.243584,0.195427,0.099907
"ALS, N=100",0.026949,0.048537,0.073420,0.073420,0.078488,0.080495,0.059907
"Own Rec, N=200",0.053468,0.080064,0.100154,0.122447,0.243584,0.195427,0.099907
"ALS, N=200",0.026949,0.048537,0.073420,0.103784,0.078488,0.080495,0.059907
"Own Rec, N=500",0.053468,0.080064,0.100154,0.122447,0.243584,0.195427,0.099907
"ALS, N=500",0.026949,0.048537,0.073420,0.103784,0.078488,0.080495,0.059907


Модель "Own Recommendations + Top Popular" дает лучше Recall, чем модель ALS. Значения recall растут с ростом k, в какой-то момент перестают расти. Для дальнейшего моделирования можно взять рекомендации модели "Own Rec, N=200" 

### Второй уровень - Ranking

**Задание 2.**

Обучите модель 2-ого уровня, при этом:

- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар

- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker

- Вырос ли precision@5 при использовании двухуровневой модели?

Нужно обучить модель 2-ого уровня на data_train_ranking, причем только на кандидатах, выбранных во время matching

Ниже будет перебор части ранее использованных рекомендательных моделей. Пользователи и их фичи не зависят от выбора модели, поэтому заранее подготовим вспомогательные датасеты.

In [13]:
# Отмечаем покупки как target=1
ranker_ones = (
    data_train_ranker[[USER_ID_COL, ITEM_ID_COL]]
    .drop_duplicates()
    .sort_values([USER_ID_COL, ITEM_ID_COL])
    .assign(target=1)
)

In [14]:
# User features
user_feature_base = (
    data_train_ranker
    .merge(item_features, on=ITEM_ID_COL, how='left')
    .merge(user_features, on=USER_ID_COL, how='left')
)

median_basket_value = (
    user_feature_base
    .groupby([USER_ID_COL, 'basket_id'])
    .agg({'sales_value': 'sum'})
    .groupby(level=0)
    .agg(median_basket_value=('sales_value', 'median'))
)

median_categories_in_basket = (
    user_feature_base
    .groupby([USER_ID_COL, 'basket_id'])
    .agg({'commodity_desc': 'nunique'})
    .groupby(level=0)
    .agg(median_categories_in_basket=('commodity_desc', 'median'))
)

week_variance = (
    user_feature_base
    .groupby([USER_ID_COL, WEEK_NUM_COL])
    .agg({'quantity': 'sum'})
    .groupby(level=0)
    .agg(week_variance=('quantity', 'var'))
)

In [15]:
%%time
# Кандидаты - результат работы одной из моделей. Попробуем разные модели
for reranking_model_name in ('ALS, N=50', 'ALS, N=200', 'Own Rec, N=200'):
    candidates = rs.result[[USER_ID_COL, reranking_model_name]]
    df_ranker_train = (
        ranker_ones[[USER_ID_COL]]
        .drop_duplicates()  # only ranker users
        .merge(candidates, on=USER_ID_COL, how='inner')
        .rename(columns={reranking_model_name: ITEM_ID_COL})
        .explode(ITEM_ID_COL)
        .merge(ranker_ones, on=[USER_ID_COL, ITEM_ID_COL], how="left")
        .fillna(0)
        .merge(item_features, on=ITEM_ID_COL, how='left')
        .merge(user_features, on=USER_ID_COL, how='left')
    )
    cat_features = [col for col in df_ranker_train.columns
                    if col not in (USER_ID_COL, ITEM_ID_COL, 'target')]
    # Additional feature generation
    # Item features
    present_items = user_feature_base[ITEM_ID_COL].unique()
    absent_items = np.setdiff1d(df_ranker_train[ITEM_ID_COL], present_items)
    additional_base = (
        data_train_matcher[data_train_matcher[ITEM_ID_COL].isin(absent_items)]
        .merge(item_features, on=ITEM_ID_COL, how='left')
        .merge(user_features, on=USER_ID_COL, how='left')
    )
    item_feature_base = pd.concat([user_feature_base, additional_base])

    median_num_per_week = (
        item_feature_base
        .groupby([ITEM_ID_COL, WEEK_NUM_COL])
        .agg({'quantity': 'sum'})
        .groupby(level=0)
        .agg(median_num_per_week=('quantity', 'median'))
    )

    average_price = (
        item_feature_base
        .assign(price = lambda x: np.where(x.quantity > 0, x.sales_value/x.quantity, np.nan))
        .groupby(ITEM_ID_COL)
        .agg(average_price=('price', np.nanmean))
        .fillna(0)
    )
    
    df_ranker_train_enriched = (
        df_ranker_train
        .merge(median_basket_value, on=USER_ID_COL, how="left")
        .merge(median_categories_in_basket, on=USER_ID_COL, how="left")
        .merge(week_variance, on=USER_ID_COL, how="left")
        .merge(median_num_per_week, on=ITEM_ID_COL, how="left")
        .merge(average_price, on=ITEM_ID_COL, how="left")
    )
    
    X_train = df_ranker_train_enriched.drop(columns=['target', USER_ID_COL, ITEM_ID_COL])
    X_train[cat_features] = X_train[cat_features].astype('category')
    y_train = df_ranker_train_enriched['target']
    lgb = LGBMClassifier(objective='binary',
                         max_depth=8,
                         n_estimators=300,
                         learning_rate=0.05).fit(X_train, y_train)
    user_item_proba = (
        df_ranker_train_enriched
        .assign(proba=lgb.predict_proba(X_train)[:,1])
        .filter([USER_ID_COL, ITEM_ID_COL, 'proba'])
    )
    rs.rerank(reranking_model_name, user_item_proba)

Wall time: 57.4 s


In [16]:
%%time
rs.get_metric_table('ranker_actual', ['recall@50', 'precision@5', 'precision@10']).sort_values('precision@5', ascending=False)

Wall time: 9.72 s


,recall@50,precision@5,precision@10
"Own Rec, N=20",0.054942,0.212783,0.166667
"Own Rec, N=50",0.082250,0.212783,0.166667
"Own Rec, N=100",0.082250,0.212783,0.166667
"Own Rec, N=200",0.082250,0.212783,0.166667
"Own Rec, N=500",0.082250,0.212783,0.166667
"Own Rec, N=200, reranked",0.063905,0.112291,0.095428
"ALS, N=50, reranked",0.052070,0.108161,0.093559
"ALS, N=200, reranked",0.054491,0.104228,0.091101
"ALS, N=20",0.028206,0.073255,0.065388
"ALS, N=50",0.052070,0.073255,0.065388


Метрика precision@5 при использовании двухуровневой модели и Own Recommendations, N=200 в качестве основной - не выросла. Должна ли она обязательно вырастать? Для ALS-моделей рост при переранжировании заметен.